In [ ]:
import openai
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import os
import os
import subprocess
import platform

import joblib

# Set your OpenAI API key
openai.api_key = "Copy Yours"

# Load and preprocess the training data
def load_training_data(train_csv_path):
    df = pd.read_csv(train_csv_path)
    prompts = df['Prompt'].values
    intents = df['Intent'].values
    return prompts, intents

# Train a Logistic Regression model
def train_logistic_model(prompts, intents):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(prompts)
    model = LogisticRegression()
    model.fit(X, intents)
    return model, vectorizer

# Train a Random Forest model
def train_random_forest_model(prompts, intents):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(prompts)
    model = RandomForestClassifier()
    model.fit(X, intents)
    return model, vectorizer

# Train an SVM model
def train_svm_model(prompts, intents):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(prompts)
    model = SVC(probability=True)  # Use probability=True for consistent output with other models
    model.fit(X, intents)
    return model, vectorizer

# Predict intent using a trained model
def predict_intent(model, vectorizer, prompt):
    X = vectorizer.transform([prompt])
    intent = model.predict(X)[0]
    return intent

# Check if intent is in the predefined list of intents
def validate_intent(intent, intent_list):
    return intent if intent in intent_list else "No Such Operation"

# Use GPT-4o or GPT-4 as a fallback
def identify_intent_with_gpt4o(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": f"Identify the operation or instruction to be performed for the given prompt using one word from the following list: Copy, list, Move, Count, Check, Rename, ipaddr, Create_file, Create_directory, remove_file, remove_directory. If the intent does not match any of these operations, display 'No Such Operation'. {prompt}"}],
        max_tokens=5,
        n=1,
        stop=None,
        temperature=0.0,
    )
    return response['choices'][0]['message']['content'].strip().lower()

#Perform the Operations

def get_ip_address():
    try:
        if platform.system() == "Linux":
            ip = subprocess.check_output("hostname -I", shell=True).decode().strip()
        elif platform.system() == "Windows":
            ip = subprocess.check_output("ipconfig", shell=True).decode()
            ip = [line.split(":")[1].strip() for line in ip.splitlines() if "IPv4 Address" in line][0]
        elif platform.system() == "Darwin":  # macOS
            ip = subprocess.check_output("ifconfig | grep inet", shell=True).decode().strip()
            ip = [line.split()[1] for line in ip.splitlines() if "inet " in line and "127.0.0.1" not in line][0]
        else:
            ip = "Unknown Operating System"
        print(f"Your IP address is: {ip}")
    except subprocess.CalledProcessError as e:
        return f"Failed to retrieve IP address: {str(e)}"
    except IndexError:
        return "Could not parse IP address from command output."

def count_files_in_directory():
    try:
        files = os.listdir('.')
        print(f"There are {len(files)} files in the current directory.")
    except Exception as e:
        print(f"Failed to count files: {str(e)}")

def create_directory():
    directory_name = input("Enter the name of the directory you want to create: ")
    try:
        os.makedirs(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
        #print("Done")
    except Exception as e:
        print(f"Failed to create directory '{directory_name}': {str(e)}")
         

def delete_directory():
    directory_name = input("Enter the name of the directory you want to delete: ")
    try:
        os.rmdir(directory_name)
        print(f"Directory '{directory_name}' deleted successfully.")
    except Exception as e:
        print(f"Failed to delete directory '{directory_name}': {str(e)}")

def list_files_in_directory():
    directory_name = input("Enter the directory name (leave blank for current directory): ") or "."
    try:
        files = os.listdir(directory_name)
        print(f"Files in '{directory_name}': {', '.join(files)}")
    except Exception as e:
        print(f"Failed to list files in directory '{directory_name}': {str(e)}")

def move_file():
    source = input("Enter the source file name: ")
    destination = input("Enter the destination path: ")
    try:
        os.rename(source, destination)
        print(f"File '{source}' moved to '{destination}'.")
    except Exception as e:
        print(f"Failed to move file '{source}': {str(e)}")

def copy_file():
    source = input("Enter the source file name: ")
    destination = input("Enter the destination path: ")
    try:
        subprocess.check_output(f"cp {source} {destination}", shell=True)
        print(f"File '{source}' copied to '{destination}'.")
    except Exception as e:
        print(f"Failed to copy file '{source}': {str(e)}")

def delete_file():
    file_name = input("Enter the name of the file you want to delete: ")
    try:
        os.remove(file_name)
        print(f"File '{file_name}' deleted successfully.")
    except Exception as e:
        print(f"Failed to delete file '{file_name}': {str(e)}")

def create_file():
    file_name = input("Enter the name of the file you want to create: ")
    try:
        with open(file_name, 'w') as file:
            print(f"File '{file_name}' Created successfully.")
    except Exception as e:
        print(f"Failed to Create file '{file_name}': {str(e)}")


def rename_file():
    old_name = input("Enter the current file name: ")
    new_name = input("Enter the new file name: ")
    try:
        os.rename(old_name, new_name)
        print(f"File '{old_name}' renamed to '{new_name}'.")
    except Exception as e:
        print(f"Failed to rename file '{old_name}': {str(e)}")

def check_file_existence():
    file_name = input("Enter the name of the file you want to check: ")
    try:
        exists = os.path.exists(file_name)
        print(f"File '{file_name}' exists: {exists}")
    except Exception as e:
        print(f"Failed to check file existence: {str(e)}")


def execute_command(command):
    if command == "ipaddr":
        return get_ip_address()
    elif command == "count":
        return count_files_in_directory()
    elif command == "create_directory":
        return create_directory()
    elif command == "remove_directory":
        return delete_directory()
    elif command == "list":
        return list_files_in_directory()
    elif command == "move":
        return move_file()
    elif command == "copy":
        return copy_file()
    elif command == "remove_file":
        return delete_file()
    elif command == "rename":
        return move_file()
    elif command == "create_file":
        return create_file()
    elif command == "check":
        return check_file_existence()
    else:
        return "Sorry, I didn't understand the command."


train_csv_path = 'Train.csv'
prompts, intents = load_training_data(train_csv_path)

# Train the Logistic Regression model
logistic_model, vectorizer_lr = train_logistic_model(prompts, intents)

# Train the Random Forest model
rf_model, vectorizer_rf = train_random_forest_model(prompts, intents)

# Train the SVM model
svm_model, vectorizer_svm = train_svm_model(prompts, intents)

# Save the models and vectorizers
joblib.dump(logistic_model, 'logistic_model.pkl')
joblib.dump(vectorizer_lr, 'vectorizer_lr.pkl')
joblib.dump(rf_model, 'rf_model.pkl')
joblib.dump(vectorizer_rf, 'vectorizer_rf.pkl')
joblib.dump(svm_model, 'svm_model.pkl')
joblib.dump(vectorizer_svm, 'vectorizer_svm.pkl')

# Load the models and vectorizers
logistic_model = joblib.load('logistic_model.pkl')
vectorizer_lr = joblib.load('vectorizer_lr.pkl')
rf_model = joblib.load('rf_model.pkl')
vectorizer_rf = joblib.load('vectorizer_rf.pkl')
svm_model = joblib.load('svm_model.pkl')
vectorizer_svm = joblib.load('vectorizer_svm.pkl')

# List of valid intents
valid_intents = ["copy", "list", "count", "move", "ipaddr", "check", "create_file", "create_directory", "remove_file", "remove_directory"]

# Main loop to take user input and process it
while True:
    user_input = input("Describe your problem or request: ")
    
    # Predict intent using Logistic Regression
    logistic_intent = predict_intent(logistic_model, vectorizer_lr, user_input)
    
    # Predict intent using Random Forest
    rf_intent = predict_intent(rf_model, vectorizer_rf, user_input)

    # Predict intent using SVM
    svm_intent = predict_intent(svm_model, vectorizer_svm, user_input)
    
    # Validate the intents
    validated_logistic_intent = validate_intent(logistic_intent, valid_intents)
    validated_rf_intent = validate_intent(rf_intent, valid_intents)
    validated_svm_intent = validate_intent(svm_intent, valid_intents)
    
    # If no valid intent, fall back to GPT-4o
    if validated_logistic_intent == "No Such Operation":
        validated_logistic_intent = identify_intent_with_gpt4o(user_input)
        
    if validated_rf_intent == "No Such Operation":
        validated_rf_intent = identify_intent_with_gpt4o(user_input)

    if validated_svm_intent == "No Such Operation":
        validated_svm_intent = identify_intent_with_gpt4o(user_input)
    
    
    # Identify intent with GPT-4o
    gpt_intent = identify_intent_with_gpt4o(user_input)
    
    print(f"Identified Intent by Logistic Regression: {validated_logistic_intent}")
    print(f"Identified Intent by Random Forest: {validated_rf_intent}")
    print(f"Identified Intent by SVM: {validated_svm_intent}")
    print(f"Identified Intent by GPT-4: {gpt_intent}")
    
    
    execute_command(gpt_intent)


Describe your problem or request:  How to know my ip address


Identified Intent by Logistic Regression: ipaddr
Identified Intent by Random Forest: ipaddr
Identified Intent by SVM: ipaddr
Identified Intent by GPT-4: ipaddr
Your IP address is: 172.27.17.40


Describe your problem or request:  Please tell my ip address


Identified Intent by Logistic Regression: ipaddr
Identified Intent by Random Forest: ipaddr
Identified Intent by SVM: ipaddr
Identified Intent by GPT-4: ipaddr
Your IP address is: 172.27.17.40


Describe your problem or request:  Need to know the ip of machine


...

Identified Intent by Logistic Regression: list
Identified Intent by Random Forest: list
Identified Intent by SVM: list
Identified Intent by GPT-4: count
There are 21 files in the current directory.


Describe your problem or request:  Display those files in the Directory


Identified Intent by Logistic Regression: list
Identified Intent by Random Forest: list
Identified Intent by SVM: list
Identified Intent by GPT-4: list


Enter the directory name (leave blank for current directory):  


Files in '.': svm_model.pkl, vectorizer_svm.pkl, .DS_Store, requirements.txt, bin, include, etc, pyvenv.cfg, rf_model.pkl, sample_revised.txt, multimodelTest.ipynb, multimodel.ipynb, vectorizer_lr.pkl, lib, readme.txt, Train.csv, .ipynb_checkpoints, vectorizer_rf.pkl, logistic_model.pkl, sample.txt, share
